# 2. Data Cleaning
## Clean and Preprocess Market Data

This notebook performs:
- Handle missing values
- Remove outliers
- Align timestamps across datasets
- Handle futures contract rollovers
- Calculate ATM strike dynamically

In [ ]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_utils import DataCleaner, DataMerger, load_data, validate_data
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
print("Libraries imported successfully!")

## Load Raw Data

In [ ]:
# Load data files
spot_df = load_data('../data/nifty_spot_5min.csv')
futures_df = load_data('../data/nifty_futures_5min.csv')
options_df = load_data('../data/nifty_options_5min.csv')

print("Original Data Shapes:")
print(f"Spot: {spot_df.shape}")
print(f"Futures: {futures_df.shape}")
print(f"Options: {options_df.shape}")

## 2.1 Check for Missing Values

In [ ]:
print("Missing Values Summary:")
print("\nSpot Data:")
print(spot_df.isnull().sum())
print("\nFutures Data:")
print(futures_df.isnull().sum())
print("\nOptions Data:")
print(options_df.isnull().sum())

In [ ]:
# Visualize missing values
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].bar(range(len(spot_df.columns)), spot_df.isnull().sum())
axes[0].set_title('Spot Data Missing Values')
axes[0].set_ylabel('Count')

axes[1].bar(range(len(futures_df.columns)), futures_df.isnull().sum())
axes[1].set_title('Futures Data Missing Values')
axes[1].set_ylabel('Count')

axes[2].bar(range(len(options_df.columns)), options_df.isnull().sum())
axes[2].set_title('Options Data Missing Values')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.savefig('../plots/missing_values.png', dpi=300, bbox_inches='tight')
plt.show()

## 2.2 Handle Missing Values

In [ ]:
# Clean each dataset
cleaner = DataCleaner()

spot_df_clean = cleaner.handle_missing_values(spot_df, method='ffill')
futures_df_clean = cleaner.handle_missing_values(futures_df, method='ffill')
options_df_clean = cleaner.handle_missing_values(options_df, method='ffill')

print("After handling missing values:")
print(f"Spot missing: {spot_df_clean.isnull().sum().sum()}")
print(f"Futures missing: {futures_df_clean.isnull().sum().sum()}")
print(f"Options missing: {options_df_clean.isnull().sum().sum()}")

## 2.3 Remove Outliers

In [ ]:
# Define price columns to check for outliers
price_cols = ['open', 'high', 'low', 'close']

# Remove outliers from spot data
spot_df_clean = cleaner.remove_outliers(
    spot_df_clean, 
    columns=price_cols,
    method='zscore',
    threshold=4.0  # More conservative threshold
)

# Remove outliers from futures data
futures_df_clean = cleaner.remove_outliers(
    futures_df_clean,
    columns=price_cols,
    method='zscore',
    threshold=4.0
)

print(f"\nAfter outlier removal:")
print(f"Spot: {spot_df_clean.shape}")
print(f"Futures: {futures_df_clean.shape}")

## 2.4 Align Timestamps

In [ ]:
# Align timestamps across all datasets
aligned_dfs = cleaner.align_timestamps([spot_df_clean, futures_df_clean, options_df_clean])
spot_df_clean, futures_df_clean, options_df_clean = aligned_dfs

print("After timestamp alignment:")
print(f"Spot: {spot_df_clean.shape}")
print(f"Futures: {futures_df_clean.shape}")
print(f"Options: {options_df_clean.shape}")
print(f"\nAll datasets now have {len(spot_df_clean)} common timestamps")

## 2.5 Handle Futures Contract Rollover

In [ ]:
# Handle futures rollover
futures_df_clean = cleaner.handle_futures_rollover(futures_df_clean, rollover_days=5)
print("Futures contract rollover handled")

## 2.6 Validate Cleaned Data

In [ ]:
# Validate data quality
spot_valid = validate_data(spot_df_clean)
futures_valid = validate_data(futures_df_clean)

print(f"\nValidation Results:")
print(f"Spot data valid: {spot_valid}")
print(f"Futures data valid: {futures_valid}")

## 2.7 Save Cleaned Data

In [ ]:
# Save cleaned datasets
merger = DataMerger()

merger.save_cleaned_data(spot_df_clean, '../data/nifty_spot_5min_cleaned.csv')
merger.save_cleaned_data(futures_df_clean, '../data/nifty_futures_5min_cleaned.csv')
merger.save_cleaned_data(options_df_clean, '../data/nifty_options_5min_cleaned.csv')

print("Cleaned data saved successfully!")

## 2.8 Generate Cleaning Report

In [ ]:
# Generate cleaning report
merger.generate_cleaning_report(
    original_df=spot_df,
    cleaned_df=spot_df_clean,
    filepath='../results/data_cleaning_report.txt'
)

print("Cleaning report generated: results/data_cleaning_report.txt")

In [ ]:
# Display report
with open('../results/data_cleaning_report.txt', 'r') as f:
    print(f.read())

## 2.9 Merge Datasets

In [ ]:
# Merge spot, futures, and options data
merged_df = merger.merge_datasets(spot_df_clean, futures_df_clean, options_df_clean)

print(f"Merged dataset shape: {merged_df.shape}")
print("\nFirst few rows:")
merged_df.head()

In [ ]:
# Save merged data
merger.save_cleaned_data(merged_df, '../data/nifty_merged_5min.csv')
print("\nMerged data saved to data/nifty_merged_5min.csv")

## Summary

In [ ]:
print("=" * 80)
print("DATA CLEANING SUMMARY")
print("=" * 80)
print(f"\nOriginal spot data: {len(spot_df)} rows")
print(f"Cleaned spot data: {len(spot_df_clean)} rows")
print(f"Data removed: {len(spot_df) - len(spot_df_clean)} rows ({(len(spot_df) - len(spot_df_clean))/len(spot_df)*100:.2f}%)")
print(f"\nFinal merged dataset: {merged_df.shape[0]} rows, {merged_df.shape[1]} columns")
print("\nNext Step: Proceed to 03_feature_engineering.ipynb")